## Getting Started with prompt Template and Chat Prompt Template
#### Use PromptTemplate to create a template for a string prompt.
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [3]:
model = GoogleGenerativeAI(model="gemini-1.5-pro-latest", google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0.5)

#### A simple string based Prompt formatting

In [ ]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt = prompt_template.format(adjective="dark", content="Finnish Winter")

response = model.invoke(prompt)

print(response)

#### ChatPromptTemplate: The prompt to chat models is a list of chat messages.
Each chat message is associated with content, and an additional parameter called role.
For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a Finnish teacher AI bot. Your name is {name}."),
    ("human", "Hello, How are you?"),
    ("ai", "Minä olen hyvaa, kiitos!"),
    ("human", "{user_input}"),
  ]
)
prompt = chat_template.format_messages(name="Nelli", user_input="What is your name? and what are you good at?")
response = model.invoke(prompt)
print(response)

#### Various ways of formatting System/Human/AI prompts

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more poetry."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
prompt = chat_template.format_messages(text="I love coding")

response = model.invoke(prompt)
print(response)

#### Providing a Context along with the Prompt

In [ ]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

response = model.invoke(prompt)
print (response)

#### Langchain’s ```FewShotPromptTemplate``` caters to source knowledge input. The idea is to “train” the model on a few examples — we call this few-shot learning — and these examples are given to the model within the prompt.

The goal of few-shot prompt templates are to dynamically select examples based on an input, and then format the example in a final prompt to provide for the model.

**Fixed Example**
The most basic (and common) few-shot prompting technique is to use a fixed prompt example.This way you can select a chain, evaluate it, and avoid worrying about additional moving parts in production.

The basic components of the template are: 
- examples: A list of dictionary examples to include in the final prompt. 
- example_prompt: converts each example into 1 or more messages through its format_messages method. A common example would be convert each example into one human message and one AI message response, or a human message followed by a function call message.

In [10]:
from langchain import FewShotPromptTemplate

# Create our examples
examples = [
  {
    "query": "How are you?",
    "answer": "I can't complain but sometimes I still do."
  },
  {
    "query": "What time is it?",
    "answer": "It's time to get a watch."
  }
]

# Create a example template
example_template = """
User: {query}
AI: {answer}
"""

In [11]:
# create a prompt example from above template

example_prompt = PromptTemplate(
  input_variables=["query", "answer"],
  template=example_template
)

# Now break our previous prompt into a prefix and suffix
# The prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to the users questions. Here are some
examples: 
"""

# And the suffix our user input and output indicator
suffix = """
user: {query}
AI:
"""

In [12]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [ ]:
query = "What movie should I watch this evening with my lover?"
print(few_shot_prompt_template.format(query=query))

In [ ]:
chain = few_shot_prompt_template | model

chain.invoke({"query": query})